In [28]:
import numpy as np
import pygame
import pandas as pd
import random
from subprocess import Popen, PIPE, STDOUT
import gymnasium as gym
from gymnasium import spaces
import time

class BasketballEnv(gym.Env):
	#metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}

	def __init__(self, render_mode=None ):
		#self.size = size  # The size of the square grid
		
		self.number_to_character = {'0' : 'Alice', '1':'Bob' , '2':'Charlie','3':'Sherlock','4':'?'}
		self.character_to_number = {v: k for k, v in self.number_to_character.items()}
													
		self.number_to_place = {'0' : 'HomeB', '1':'BasketballCourt' , '2':'Downtown','3':'?'}
		self.place_to_number = {v: k for k, v in self.number_to_place.items()}

		self.number_to_item = {'0' : 'Basketball', '1':'Bat','2':'?' }
		self.item_to_number = {v: k for k, v in self.number_to_item.items()}

		self.number_to_crime = {'0' : 'Theft', '1':'Murder','2':'?' }
		self.crime_to_number = {v: k for k, v in self.number_to_crime.items()}

		
		#self._nb_features = 39
		self.score = 0
		self.characters = ['Alice', 'Bob', 'Charlie','Sherlock']
		self.items = ['Basketball','Bat']
		self.places= ['HomeB','BasketballCourt','Downtown']
		self.crimes = ['Theft','Murder']

		self.utilities = {
			'Alice':'1 - angry(Alice);\n',
			'Bob':'3 - (sum(c : citizen) angry(c));\n',
			'Charlie':'if(!alive(Alice)) 1 else 0;\n',
			'Sherlock':'(sum(c : citizen) underArrest(c)); \n',
		}

		self.utilities1 = {
			'Alice':'1 - angry(Alice);\n',
			'Bob':'3 - (sum(c : citizen) angry(c));\n',
			'Charlie':'if(!alive(Alice)) 1 else 0;\n',
			'Sherlock':'(sum(c : citizen) underArrest(c)); \n',
		}

		self.utilities2 = {
			'Alice':'1 - angry(Alice);\n',
			'Bob':'3 - (sum(c : citizen) angry(c));\n',
			'Charlie':'if(!alive(Alice)) 1 else 0;\n',
			'Sherlock':' (sum(p : place) searched(p)); \n',


		}
		"""
		self.characters = ['0','1','2','3']
		self.items = ['0','1']
		self.places= ['0','1','2']
		self.crimes = ['0','1']
		self.utilities = {
			'0':'1 - angry(Alice);\n',
			'1':'3 - (sum(c : citizen) angry(c));\n',
			'2':'if(!alive(Alice)) 1 else 0;\n',
			'3':'(sum(c : citizen) underArrest(c)) + (sum(p : place) searched(p)); \n',
		}
		"""
		self.acting_character = self.characters[0]
		self.additional_utility = None
		self.df_effects = pd.read_csv('basketball_effects_nn.csv')
		self.file = 'rl_planner.txt'
		
		self.randomize_df()
		self.df = self.change_number_to_cat(self.df)
		self.create_file(self.df)
		self.no_solution = 0
		self._nb_features = len(self.df.index)
		# Observations are dictionaries with the agent's and the target's location.
		# Each location is encoded as an element of {0, ..., `size`}^2, i.e. MultiDiscrete([size, size]).
		self.observation_space = spaces.Box(
			0,
			5,
			shape = [self._nb_features]
		)

		self.target_feature = None
		self.target_value = None

		

		# We have 4 actions, corresponding to choosing character
		self.action_space = spaces.Discrete(6)

		"""
		The following dictionary maps abstract actions from `self.action_space` to
		the direction we will walk in if that action is taken.
		
		"""
		self._action_to_direction = {
			0: 'char0',
			1: 'char1',
			2: 'char2',
			3: 'char3',
			4: 'Sherlock_Utility_1',
			5: 'Sherlock_Utility_2'
		}

		print('nb',len(self.df.index))
		

		

	def change_number_to_cat(self,row):
		for index,value in row.items():
			if ('acting_character' in index)  or ('has' in index) :
				
				row[index] = self.number_to_character[str(value)]
			
			elif 'at' in index:
				row[index] = self.number_to_place[str(value)]

		return row

	def change_cat_to_number(self,row):
		for index,value in row.items():
			if ('acting_character' in index) or ('has' in index):
				if str(value) in self.character_to_number.keys():
					row[index] = self.character_to_number[str(value)]
			
			elif 'at' in index:
				if str(value) in self.place_to_number.keys():
					row[index] = self.place_to_number[str(value)]

		return row


	def randomize_df(self):
		

		lists = []
		columns = []

		columns.append("acting_character")
		lists.append(self.characters)

		for c in self.characters:
			c= str(c)
			columns.append(c+"_alive")
			lists.append([0,1])

			columns.append(c+"_underArrest")
			lists.append([0,1])

			columns.append(c+"_angry")
			lists.append([0,1])

			columns.append(c+"_suspect")
			lists.append(self.crimes)

			columns.append(c+"_at")
			lists.append(self.places)
		
			
		for p in self.places:
			p = str(p)
			columns.append(p+"_searched")
			lists.append([0,1])

		characters2 = self.characters + ['?']
		for i in self.items:
			i = str(i)
			columns.append(i+"_has")
			lists.append(characters2 )

		for c in self.crimes:
			for i in self.items:
				for p in self.places:
					columns.append(str(c)+"_"+str(i)+"_"+ str(p) +"_clues")
					lists.append([0,0,0,0,0,0,0,0,0,0,0,1])
		

		d = []
		for l in lists:
			d.append(random.choice(l))

		#print(columns)
		d = ['0','1','0','1','2','2','1','0','0','2','0','1','0','1','2','2','1','0','0','2','2','0','0','0','1','2','0','0','0','0','0','0','0','0','0','0','0','0']
		self.df = pd.Series(data = d, index=columns )
		
		
		
	def create_file(self,row):
		acting_character = self.acting_character
		with open(self.file, 'w') as f:
			f.write("""type item;
type place;
type basketballPlace : place;
type arrestPlace : place;
type crime; 
type citizen : character;
type police : character;
type detective : police;
type inspector : police;

property alive(character : character) : boolean;
property underArrest(character : character) : number;
property angry(character : character) : number;
property searched(place : place) : number;
property suspect(character : character, c : crime) : boolean;
property clue(crime : crime, item : item, place : place) : boolean;
property at(character : character) : place;
property has(item : item) : character;

entity Alice : citizen;
entity Bob : citizen;
entity Charlie : citizen;
entity Sherlock : detective;
entity HomeB : place;
entity BasketballCourt : basketballPlace;
entity Downtown : arrestPlace;
entity Basketball : item;
entity Bat : item;
entity Theft : crime;
entity Murder : crime;

		   """
			)
			
			for char in self.characters:
				if str(row[char+"_alive"]) == '1':
					f.write("alive(" + char  +")  ;\n")
				f.write("underArrest(" + char  +") = " + str(row[char+"_underArrest"]) + " ;\n")
				f.write("angry(" + char  +") = " + str(row[char+"_angry"]) + " ;\n")
				if str(row[char+"_suspect"]) != '2':
					f.write("suspect(" + char + ', '+ str(row[char+"_suspect"]) +")   ;\n")
				f.write("at(" + char  +") = " + str(row[char+"_at"]) + " ;\n")

			
				
			for p in self.places:
				f.write("searched(" + p  +") = " + str(row[p+"_searched"]) + " ;\n")
				

			
			for i in self.items:
				f.write("has(" + i  +") = " + str(row[i+"_has"]) + " ;\n")
				
			clues = [x for x in row.index if 'clues' in x]
			for c in clues:
				if str(row[c]) == '1':
					entities = c.split('_')
					f.write("clue(" + entities[0]+","+entities[1]+","+ entities[2] +")   ;\n")
				#warrtosci postaci
			

				
				

			f.write("""
action travel(character : character, from : place, to : place){
	precondition:
		from != to & 
		at(character) == from &
		alive(character);
	effect:
		at(character) = to;
	consenting: character; 
	observing(c : character) : at(c) == from | at(c) == to; 
};

action arrest(police : police, character : character, place : place, crime : crime){
	precondition: 
		at(police) == place &
		at(character) == place &
		police != character &
		alive(police) &
		alive(character) &
		suspect(character, crime);
	effect:
		underArrest(character) = 1;
	consenting: police;
	observing(a : character) : at(a) == place;
};

action steal(thief : citizen, victim : citizen, item : item, place : place){
	precondition:
		at(thief) == place &
		at(victim) == place &
		has(item) == victim &
		thief != victim &
		alive(thief);
	effect:
		has(item) = thief &
		angry(victim) = 1 &
		clue(Theft, item, place);
	consenting: thief;
	observing(c : character) : (c == thief | c == victim) | (at(c) == place & place != Downtown); // crimes downtown aren't observed
};

action play_basketball(player1 : citizen, player2 : citizen, place : basketballPlace){
	precondition:
		player1 != player2 &
		at(player1) == place &
		alive(player1) &
		at(player2) == place &
		alive(player2) &
		has(Basketball) == player1;
	effect:
		angry(player1) = 0 &
		angry(player2) = 0;
	consenting: player1, player2;
	observing(c : character) : at(c) == place;
};

action kill(killer : citizen, victim : citizen, item : item, place : place){
	precondition:
		killer != victim &
		at(killer) == place &
		at(victim) == place &
		alive(killer) &
		alive(victim) &
		has(item) == killer &
		underArrest(killer) == 0;
	effect:
		!alive(victim) &
		clue(Murder, item, place);
	consenting: killer;
	observing(c : character) : c == killer | (at(c) == place & place != Downtown); 
};
	
action find_clues(police : police, crime : crime, item : item, place : place){
	precondition:
		at(police) == place &
		alive(police)
		&clue(crime, item, place);
	effect:
		searched(place) = 1 &
		if(clue(crime, item, place))
			believes(police, clue(crime, item, place));
	consenting: police;
	observing(c : character) : at(c) == place;
};

action share_clues(police1 : police, police2 : police, crime : crime, item : item, place : place){
	precondition:
		police1 != police2 &
		at(police1) == place &
		alive(police1) &
		at(police2) == place &
		alive(police2) &
		clue(crime, item, place);
	effect:
		believes(police2, clue(crime, item, place));
	consenting: police1;
	observing(c : character) : at(c) == place;
};
 
action suspect_of_crime(police : police, citizen : citizen, crime : crime, item : item, place : place){
	precondition:
		police != citizen &
		at(police) == place &
		alive(police) &
		at(citizen) == place &
		alive(citizen) &
		has(item) == citizen &
		exists(p : place) clue(crime, item, p);
	effect:
		suspect(citizen, crime);
	consenting: police;
	observing(c : character) : at(c) == place;
};

trigger see_has(character : character, other : character, item : item, place : place){
	precondition:
		at(character) == place &
		at(other) == place &
		has(item) == other &
		believes(character, has(item) != other);
	effect:
		believes(character, has(item) = other);
};

trigger see_hasnt(character : character, other : character, item : item, place : place){
	precondition:
		at(character) == place &
		at(other) == place &
		has(item) != other & 
		believes(character, has(item) == other);
	effect:
		believes(character, has(item) = ?);
};

trigger see_at(character : character, other : character, place : place){
	precondition:
		at(character) == place &
		at(other) == place &
		believes(character, at(other) != place);
	effect:
		believes(character, at(other) = place);
};

trigger see_gone(character : character, other : character, place : place){
	precondition:
		at(character) == place &
		at(other) != place &
		believes(character, at(other) == place);
	effect:
		believes(character, at(other) = ?);
};

			""")
		   #.format(acting_character)

			
			
			f.write("utility(): \n ")
			f.write(self.utilities[acting_character])

			
		


			for char in self.characters:
				f.write("utility({}): \n ".format(char))
				f.write(self.utilities[char])
			
			
			f.close()


	def _get_obs(self):
		
		df2 = self.change_cat_to_number(self.df) 
		r = df2.to_numpy(dtype='float32')
		#print(len(r))
		return r
	
	def _get_info(self):
		return {0:'test'}

	def reset(self, seed=None, options=None):
		# We need the following line to seed self.np_random
		super().reset(seed=seed)
		print('reset!')
		#randomize row
		self.randomize_df()
		self.df = self.change_number_to_cat(self.df)
		self.create_file(self.df)
		self.no_solution = 0
		self.score = 0
		self.acting_character = self.characters[0]
		observation = self._get_obs() #turn row into observation
		info = self._get_info() #turn row into info
		self.utilities = self.utilities1
		if self.render_mode == "human":
			self._render_frame()

		return observation, info
	
	
	def load_action(self,file):

		
		p = Popen(['java', '-jar', '..\lib\sabre.jar', '-p', file,'-el',"0","-h","h+",'-c','n',"-tl","5000"], stdout=PIPE, stderr=STDOUT)
		#p = Popen(['java', '-jar', 'lib\sabre.jar', '-p', file,'-el',"0",'-g',"","-tl","1000"], stdout=PIPE, stderr=STDOUT)

		lines=[]
		for line in p.stdout:
			lines.append(str(line, encoding='utf-8'))

		#print(lines)
		return lines[0].replace("\r\n","")

	def do_action(self,args):
		if len(args) > 0 and len(self.df_effects[self.df_effects.action == args[0] ]['effect_function'].values) > 0:
			functions = self.df_effects[self.df_effects.action == args[0] ]['effect_function'].values[0].split(';')
			for function in functions:
				parts = function.split(':')
				
				
				#choosing feature
				feature = parts[0]

				#choosing how feature is changed
				change = parts[1]

				for count,arg in enumerate(args,0):
					
						feature = feature.replace('arg'+str(count),arg)
					
						change = change.replace('arg'+str(count),arg)

				change = change.split("_")
				print(feature)
				print(change)

				

				
				
				if change[0] == "=":
					print("!")
					print(change[1])
					self.df[feature] = change[1]
					print(self.df[feature])
				elif change[0] == "+":
					self.df[feature] = int(self.df[feature].values[0]) + int(change[1])
				elif change[0] == "-":
					self.df[feature] = int(self.df[feature]) - int(change[1])
		
		
				#print(self.df.index)
				self.df = self.change_cat_to_number(self.df)
			

	def change_state(self,actions):
		
		
		
		
		if 'No solution' not in actions:
			actions = actions.split(') ')
			
			if len(actions) > 0:
				args = actions[0].replace("("," ").replace(")","").replace(",","")
				print(args)
				if ('key_action' not in args) :
					self.do_action(args.split(" "))
				
				return True
		
		return False

	def make_action(self,action):
		if 'char' in action:
			char = action.split('char')[1]
			self.acting_character = self.number_to_character[str(char)]
			print(self.acting_character)
		elif 'Sherlock_Utility_1' in action:
			self.utilities = self.utilities1
			#self.additional_utility = action
		elif 'Sherlock_Utility_2' in action:
			self.utilities = self.utilities2
		else: print("nie ma takiej akcji!")
		
		
		self.create_file(self.change_number_to_cat(self.df))
		start = time.time()

		
		
		self.results = self.load_action(self.file)
		

		end = time.time()
		
		index = 0
		#self.df.loc[index,['results']] = self.results
		#self.df.loc[index,['time']] = (end-start)

		print(self.results)
		self.change_state(self.results)
		
	def get_reward(self):
		#stworzenie targetu na poczatku
		reward = 0
		#czy osiagnelismy target jesli tak to 1
		sum = int(self.df['Alice_underArrest']) + int(self.df['Bob_underArrest']) + int(self.df['Charlie_underArrest'])
		if self.df['Charlie_underArrest'] == '1':
			reward = 1
		elif 'No solution' in self.results or 'Time limit' in self.results:
			reward = -0.25
			self.no_solution = self.no_solution+1
		else: 
			self.no_solution = 0
			reward = -1
		#jesli nie to 0
		#jesli nie działa to minus
		#jesli to działa, to ustawienie samemu targetu, też w tabeli
		
		return reward

	def step(self, action):
		# Map the action (element of {0,1,2,3}) to the direction we walk in
		direction = self._action_to_direction[action]
		
		self.make_action(direction)

		
		terminated = False
		reward = self.get_reward()
		if reward > 0 :
			terminated = True
		if self.no_solution > 6:
			terminated = True
		self.score += reward
		observation = self._get_obs()
		info = self._get_info()

		#if self.render_mode == "human":
			

		return observation, reward, terminated, self.score, info

	def render(self):
		print(self.df)

In [29]:
BasketballEnv()

nb 38


In [25]:
class PrioritizedReplayBuffer:
    def __init__(self, capacity, epsilon=1e-6, alpha=0.8, beta=0.4, beta_increment=0.001):
        self.capacity = capacity
        self.epsilon = epsilon
        self.alpha = alpha   # how much prioritisation is used
        self.beta = beta    # for importance sampling weights
        self.beta_increment = beta_increment
        self.priority_buffer = np.zeros(self.capacity)
        self.data = []
        self.position = 0

    def length(self):
        return len(self.data)

    def push(self, experience):
        max_priority = np.max(self.priority_buffer) if self.data else 1.0
        if len(self.data) < self.capacity:
            self.data.append(experience)
        else:
            self.data[self.position] = experience
        self.priority_buffer[self.position] = max_priority
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        priorities = self.priority_buffer[:len(self.data)]
        probabilities = priorities ** self.alpha
        probabilities /= probabilities.sum()

        indices = np.random.choice(len(self.data), batch_size, p=probabilities)
        experiences = [self.data[i] for i in indices]

        total = len(self.data)
        weights = (total * probabilities[indices]) ** (-self.beta)
        weights /= weights.max()

        self.beta = np.min([1., self.beta + self.beta_increment])
        
        return experiences, indices, weights

    def update_priorities(self, indices, errors):
        for idx, error in zip(indices, errors):
            self.priority_buffer[idx] = error + self.epsilon

In [26]:
import tensorflow as tf

class DQN:
    def __init__(self, state_shape, action_size, learning_rate_max=0.001, learning_rate_decay=0.995, gamma=0.75, 
                 memory_size=2000, batch_size=32, exploration_max=1.0, exploration_min=0.01, exploration_decay=0.995):
        self.state_shape = state_shape
        self.state_tensor_shape = (-1,) + state_shape
        self.action_size = action_size
        self.learning_rate_max = learning_rate_max
        self.learning_rate = learning_rate_max
        self.learning_rate_decay = learning_rate_decay
        self.gamma = gamma
        self.memory_size = memory_size
        self.memory = PrioritizedReplayBuffer(capacity=2000)
        self.batch_size = batch_size
        self.exploration_rate = exploration_max
        self.exploration_max = exploration_max
        self.exploration_min = exploration_min
        self.exploration_decay = exploration_decay
        
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _build_model(self):
        # the actual neural network structure
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Input(shape=self.state_shape))
        model.add(tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
        model.add(tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
        model.add(tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
        model.add(tf.keras.layers.Dropout(0.1))
        model.add(tf.keras.layers.Dense(self.action_size, activation='linear', name='action_values', kernel_initializer='he_uniform'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model
    
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.push((state, action, reward, next_state, done))

    def act(self, state, epsilon=None):
        if epsilon == None:
            epsilon = self.exploration_rate
        if np.random.rand() < epsilon:
            return random.randrange(self.action_size)
        return np.argmax(self.target_model.predict(state, verbose=0)[0])
    
    def replay(self, episode=0):

        if self.memory.length() < self.batch_size:
            return None
        
        experiences, indices, weights = self.memory.sample(self.batch_size)
        unpacked_experiences = list(zip(*experiences))
        states, actions, rewards, next_states, dones = [list(arr) for arr in unpacked_experiences]

        # Convert to tensors
        states = tf.convert_to_tensor(states)
        states = tf.reshape(states, self.state_tensor_shape)
        actions = tf.convert_to_tensor(actions, dtype=tf.int32)
        rewards = tf.convert_to_tensor(rewards, dtype=tf.float32)
        next_states = tf.convert_to_tensor(next_states)
        next_states = tf.reshape(next_states, self.state_tensor_shape)
        dones = tf.convert_to_tensor(dones, dtype=tf.float32)

        # Compute Q values and next Q values
        target_q_values = self.target_model.predict(next_states, verbose=0)
        q_values = self.model.predict(states, verbose=0)

        # Compute target values using the Bellman equation
        max_target_q_values = np.max(target_q_values, axis=1)
        targets = rewards + (1 - dones) * self.gamma * max_target_q_values

        # Compute TD errors
        batch_indices = np.arange(self.batch_size)
        q_values_current_action = q_values[batch_indices, actions]
        td_errors = targets - q_values_current_action
        self.memory.update_priorities(indices, np.abs(td_errors))

        # For learning: Adjust Q values of taken actions to match the computed targets
        q_values[batch_indices, actions] = targets

        loss = self.model.train_on_batch(states, q_values, sample_weight=weights)

        self.exploration_rate = self.exploration_max*self.exploration_decay**episode
        self.exploration_rate = max(self.exploration_min, self.exploration_rate)
        self.learning_rate = self.learning_rate_max*self.learning_rate_decay**episode
        tf.keras.backend.set_value(self.model.optimizer.learning_rate, self.learning_rate)

        return loss
    
    def load(self, name):
        self.model = tf.keras.models.load_model(name)
        self.target_model = tf.keras.models.load_model(name)

    def save(self, name):
        self.model.save(name)

In [27]:
import numpy as np
from collections import deque



LEARNING_RATE = 1e-4
LEARNING_RATE_DECAY = 0.99
EXPLORATION_DECAY = 0.95
GAMMA = 0.975
UPDATE_TARGET_EVERY = 10

BATCH_SIZE = 128
EPISODES = 101

env = BasketballEnv() 
state, info = env.reset()



agent = DQN(
    state_shape=state.shape,
    action_size=env.action_space.n,
    batch_size=BATCH_SIZE,
    learning_rate_max=LEARNING_RATE,
    learning_rate_decay=LEARNING_RATE_DECAY,
    exploration_decay=EXPLORATION_DECAY,
    gamma=GAMMA
)
agent.save(f'models/-1.h5')

state = env.reset()
state = np.expand_dims(state, axis=0)

most_recent_losses = deque(maxlen=BATCH_SIZE)

log = []

# fill up memory before training starts
while agent.memory.length() < BATCH_SIZE:
    action = agent.act(state)
    observation, reward, done, score, info = env.step(action)
    next_state = np.expand_dims(next_state, axis=0)
    agent.remember(state, action, reward, next_state, done)
    state = next_state

for e in range(0, EPISODES):
    state = env.reset()
    state = np.expand_dims(state, axis=0)
    done = False
    step = 0
    ma_loss = None

    while not done:
        action = agent.act(state)
        observation, reward, done, score, info = env.step(action)
        next_state = np.expand_dims(next_state, axis=0)
        agent.remember(state, action, reward, next_state, done)

        state = next_state
        step += 1

        loss = agent.replay(episode=e)
        most_recent_losses.append(loss)
        ma_loss = np.array(most_recent_losses).mean()

        if loss != None:
            print(f"Step: {step}. Score: {score}. -- Loss: {loss}", end="          \r")

        if done:
            print(f"Episode {e}/{EPISODES-1} completed with {step} steps. Score: {score:.0f}. LR: {agent.learning_rate:.6f}. EP: {agent.exploration_rate:.2f}. MA loss: {ma_loss:.6f}")
            break

    log.append([e, step, score, agent.learning_rate, agent.exploration_rate, ma_loss])

    agent.save(f'models/{e}.h5')

reset!
reset!


C:\Users\kacpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\shape_base.py:591: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = asanyarray(a)


Alice
Time limit reached.
Time limit reached.


NameError: name 'next_state' is not defined